In [1]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import gzip
from datetime import datetime, timedelta
from statistics import mean
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization, ReLU
from tensorflow.keras.activations import sigmoid, tanh
from tensorflow.keras.utils import plot_model

from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm
import csv
import random

In [2]:
def retrieve_data(varname, filename):
    df = pd.read_csv(filename)
    df["Date"] = pd.to_datetime(df["Date"])
    return df

def create_classification_data(df, lookback):
    rows = []
    columns = ['Date', 'SP500_relative_change_perc_1'] # date and target
    for i in range(1, lookback + 1):
        new_columns = df.columns.tolist()[1:]
        for x in range(len(new_columns)):
            new_columns[x] = new_columns[x] + "_t-" + str(i)
        columns = columns + new_columns
        
    for i, row in enumerate(df.iterrows()):
        if i > lookback:
            new_row = [row[1][0], row[1][2]]
            for x in range(1, lookback + 1):
                add_row = df.iloc[i - x].tolist()
                new_row = new_row + add_row[1:]
            rows.append(new_row)
    df2 = pd.DataFrame(rows)
    df2.columns = columns
    return df2

def create_train_val_test(df, year_val, year_test, perc_train=None):
    if perc_train == None:
        val = df[df['Date'].dt.year == year_val]
        test = df[df['Date'].dt.year == year_test]
        train = df[df['Date'].dt.year != year_val]
        train = train[train['Date'].dt.year != year_test]
    else:
        train = df.head(round(len(df) * perc_train))
        val = df.tail(len(df) - len(train))
        test = val.tail(round(0.5 * len(val)))
        val = df.head(len(val) - len(test))
    
    y_train = train['SP500_relative_change_perc_1']
    x_train = train.drop(['SP500_relative_change_perc_1'], axis=1)
    
    y_val = val['SP500_relative_change_perc_1']
    x_val = val.drop(['SP500_relative_change_perc_1'], axis=1)
    
    y_test = test['SP500_relative_change_perc_1']
    x_test = test.drop(['SP500_relative_change_perc_1'], axis=1)
    
    return x_train, y_train, x_val, y_val, x_test, y_test

def scale_data(x):
    standard_scaler = MinMaxScaler()
    x = x.drop(["Date"], axis=1)
    x_scaled = pd.DataFrame(standard_scaler.fit_transform(x), columns=x.columns)
    return x_scaled

In [11]:
class AE(object):
    def __init__(self, x, activation_functions, batch_sizes):
        global x_train
        global x_val
        global epochs
        
        print(f"x {x}")
        
        self.x_train = x_train
        self.x_val = x_val
        self.epochs = epochs
        
        self.activation_function = activation_functions[x[0]]
        
        self.nodes1 = x[1]
        self.nodes2 = x[2]
        self.nodes3 = x[3]
        
        self.encoding_size = x[4]
        
        self.batch_normalization = x[5]
        self.batch_size = batch_sizes[x[6]]
        
    def fit(self):
        input_shape = x_train.shape[1]
        visible = Input(shape=(input_shape,))
        
        # ENCODER
        # encoder layer 1
        encoder = Dense(self.nodes1)(visible)

        # apply batch normalization
        if self.batch_normalization == 1:
            encoder = BatchNormalization()(encoder)

        # activate layer
        if self.activation_function == "LeakyReLU":
            encoder = LeakyReLU()(encoder)
        elif self.activation_function == "ReLU":
            encoder = ReLU()(encoder)
        elif self.activation_function == "sigmoid":
            encoder = sigmoid(encoder)
        elif self.activation_function == "tanh":
            encoder = tanh(encoder)
        
        # encoder layer 2
        if self.nodes2 > 0:
            encoder = Dense(self.nodes2)(encoder)
            
            # apply batch normalization
            if self.batch_normalization == 1:
                encoder = BatchNormalization()(encoder)
            
            # activate layer
            if self.activation_function == "LeakyReLU":
                encoder = LeakyReLU()(encoder)
            elif self.activation_function == "ReLU":
                encoder = ReLU()(encoder)
            elif self.activation_function == "sigmoid":
                encoder = sigmoid(encoder)
            elif self.activation_function == "tanh":
                encoder = tanh(encoder)
        
        # encoder layer 3
        if self.nodes3 > 0:
            encoder = Dense(self.nodes3)(encoder)
            
            # apply batch normalization
            if self.batch_normalization == 1:
                encoder = BatchNormalization()(encoder)
                
            # activate layer
            if self.activation_function == "LeakyReLU":
                encoder = LeakyReLU()(encoder)
            elif self.activation_function == "ReLU":
                encoder = ReLU()(encoder)
            elif self.activation_function == "sigmoid":
                encoder = sigmoid(encoder)
            elif self.activation_function == "tanh":
                encoder = tanh(encoder)
        
        # ENCODING
        encoding = Dense(self.encoding_size)(encoder) 
        
        # DECODER
        # decoder layer 1
        if self.nodes3 > 0:
            decoder = Dense(self.nodes3)(encoding)
            
            # apply batch normalization
            if self.batch_normalization == 1:
                decoder = BatchNormalization()(decoder)
                
            # activate layer
            if self.activation_function == "LeakyReLU":
                decoder = LeakyReLU()(decoder)
            elif self.activation_function == "ReLU":
                decoder = ReLU()(decoder)
            elif self.activation_function == "sigmoid":
                decoder = sigmoid(decoder)
            elif self.activation_function == "tanh":
                decoder = tanh(decoder)
        
        # decoder layer 2
        if self.nodes2 > 0:
            if self.nodes3 == 0:
                decoder = Dense(self.nodes2)(encoding)
            else:
                decoder = Dense(self.nodes2)(decoder)
            
            # apply batch normalization
            if self.batch_normalization == 1:
                decoder = BatchNormalization()(decoder)
                
            # activate layer
            if self.activation_function == "LeakyReLU":
                decoder = LeakyReLU()(decoder)
            elif self.activation_function == "ReLU":
                decoder = ReLU()(decoder)
            elif self.activation_function == "sigmoid":
                decoder = sigmoid(decoder)
            elif self.activation_function == "tanh":
                decoder = tanh(decoder)
        
        # decoder layer 3
        if self.nodes2 == 0 and self.nodes3 == 0:
            decoder = Dense(self.nodes1)(encoding)
        else:
            decoder = Dense(self.nodes1)(decoder)

        # apply batch normalization
        if self.batch_normalization == 1:
            decoder = BatchNormalization()(decoder)

        # activate layer
        if self.activation_function == "LeakyReLU":
            decoder = LeakyReLU()(decoder)
        elif self.activation_function == "ReLU":
            decoder = ReLU()(decoder)
        elif self.activation_function == "sigmoid":
            decoder = sigmoid(decoder)
        elif self.activation_function == "tanh":
            decoder = tanh(decoder)
            
        output = Dense(input_shape, activation='linear')(decoder)
        
        model = Model(inputs=visible, outputs=output)
        model.compile(optimizer='adam', loss='mse')
        
        history = model.fit(self.x_train, self.x_train, 
                    epochs=self.epochs, batch_size=self.batch_size, verbose=0, 
                    validation_data=(self.x_val, self.x_val))
        
        train_loss = history.history['loss'][-1]
        val_loss = history.history['val_loss'][-1]
        return train_loss, val_loss

In [21]:
def calculate_fitness(x, train_loss, val_loss, input_size):
    """
    nodes1 = x[1]
    nodes2 = x[2]
    nodes3 = x[3]

    encoding_size = x[4] # goed als het kleiner is
    
    fitness = val_loss * (nodes1 + 2 * nodes2 + 3 * nodes3) * (encoding_size / input_size)
    """
    nodes1 = x[1]
    nodes2 = x[2]
    nodes3 = x[3]

    encoding_size = x[4] # goed als het kleiner is
    
    fitness = (val_loss ** 3) * (nodes1 + 2 * nodes2 + 3 * nodes3) * max(0.1, (encoding_size / input_size))
    return fitness

In [26]:
class EA(object):
    def __init__(self, population_size, activation_functions, batch_sizes, variables):
        self.population_size = population_size
        self.a = 0.2
        self.activation_functions = activation_functions
        self.batch_sizes = batch_sizes
        self.variables = variables
        
    def evaluate(self, x):
        """
        include in fitness function
            relative difference between train_loss and val_loss (smaller is better)
            number of layers (smaller is better)
            bottleneck size (smaller is better)
            val_loss (smaller is better)
        
        """
        ae = AE(x, activation_functions, batch_sizes)
        train_loss, val_loss = ae.fit()
        fitness = calculate_fitness(x, train_loss, val_loss, self.variables)
        return fitness
    
    def select_triple(self, candidate, population):
        # select three random instances for differential evolution
        x1, x2, x3 = np.random.choice(range(len(population))), np.random.choice(range(len(population))), np.random.choice(range(len(population)))
        while candidate == x1 or candidate == x2 or candidate == x3 or x1 == x2 or x2 == x3 or x1 == x3:
            # keep selecting new ones until candidate != x1 != x2 != x3
            x1, x2, x3 = np.random.choice(range(len(population))), np.random.choice(range(len(population))), np.random.choice(range(len(population)))
        return population[x1], population[x2], population[x3]
    
    def mutate(self, x1, x2, x3):
        mutated = x1 + (self.a * (x3 - x2))
        print(f"mutated {mutated}")
        # activation function
        mutated[0] = round(mutated[0])
        mutated[0] = min(mutated[0], len(self.activation_functions) - 1)
        mutated[0] = max(0, mutated[0])

        # nodes layer 1
        mutated[1] = round(mutated[1])
        mutated[1] = max(1, mutated[1]) # must be at least one

        # nodes layer 2
        mutated[2] = round(mutated[2])
        mutated[2] = max(0, mutated[2])

        # nodes layer 3
        mutated[3] = round(mutated[3])
        mutated[3] = max(0, mutated[3])

        # encoding size
        mutated[4] = round(mutated[4])
        mutated[4] = max(0, mutated[4])
        mutated[4] = min(mutated[4], self.variables) # must be smaller than or equal to the number of variables  
        mutated[4] = min(mutated[4], self.variables) # must be smaller than or equal to the number of variables  

        # batch normalization  
        mutated[5] = round(mutated[5])
        mutated[5] = max(0, mutated[5])
        mutated[5] = min(mutated[5], 1)

        # batch size
        mutated[6] = round(mutated[6])
        mutated[6] = min(mutated[6], len(self.batch_sizes) - 1)
        mutated[6] = max(0, mutated[6])
        return mutated
        
    def recombine(self, candidate, mutation):
        for i in range(candidate.shape[0]):
            prob = np.random.randint(0, 2)
            if prob == 1:
                candidate[i] = mutation[i]
        print(f"candidate {candidate}")
        # nodes 3 can only be non-zero if nodes 2 is non-zero
        if candidate[2] == 0:
            candidate[3] = 0
        # nodes per layer must be at least as large as the encoding size (or 0 for nodes2 and nodes3)
        candidate[4] = max(0.1 * self.variables, candidate[4])
        candidate[1] = max(candidate[1], candidate[4])

        if candidate[2] < candidate[4]:
            if candidate[2] <= 0.5 * candidate[4]:
                candidate[2] = 0
            else:
                candidate[2] = candidate[4]
        if candidate[3] < candidate[4]:
            if candidate[3] <= 0.5 * candidate[4]:
                candidate[3] = 0
            else:
                candidate[3] = candidate[4]
        return candidate

    def select(self, x_new, f_new, x_old, f_old):
        x_cat = np.concatenate([x_new, x_old], 0)
        f_cat = np.concatenate([f_new, f_old])
        ind = np.argsort(f_cat)
        x = x_cat[ind]
        f = f_cat[ind]
        return x[:self.population_size], f[:self.population_size]
    
    def step(self, x_old, f_old):
        x = np.copy(x_old)
        f = np.copy(f_old)
        for i in tqdm(range(self.population_size), total=self.population_size):
            # choose candidate
            candidate = x[i]
            # select 3 instances for differential evolution
            x1, x2, x3 = self.select_triple(i, x)
            # mutate 3 instances
            mutated_triple = self.mutate(x1, x2, x3)
            # recombine candidate with mutation
            candidate = self.recombine(candidate, mutated_triple)
            x[i] = candidate
            # evaluate candidate solution
            f_candidate = self.evaluate(candidate)
            f[i] = f_candidate
        # select survivors
        x, f = self.select(x, f, x_old, f_old)
        return x, f

In [27]:
with open('AE output/optimization_output_15_04_2022.csv', mode='w', newline='') as output_file:
    csv_writer = csv.writer(output_file, delimiter=',', quotechar='"')
    row = ['activation_function', 'layer1', 'layer2', 'layer3', 'encoding_size', 
            'batch_normalization', 'batch_size']
#           'train_loss', 'val_loss', 'fitness']
    csv_writer.writerow(row)

In [30]:
def init_population(population_size, activation_functions, batch_sizes, variables):
    # generate initial population
    population = []
    print("Creating initial population...")
    for i in tqdm(range(population_size), total=population_size):
        activation_function = random.randint(0, len(activation_functions) - 1)
        encoding_size = random.randint(0.1 * variables, variables)
        nodes1 = random.randint(encoding_size, 2 * variables)
        nodes2 = random.randint(encoding_size, 2 * variables)
        nodes3 = random.randint(encoding_size, 2 * variables)
        batch_normalization = random.randint(0,1)
        batch_size = random.randint(0, len(batch_sizes) - 1)
    
        population.append(np.asarray([activation_function, nodes1, nodes2, nodes3, encoding_size, batch_normalization, batch_size], dtype='object'))
    print("Initial population ready")
    return np.asarray(population)

def evaluate_init_population(ea, x):
    # evaluate initial population
    f = []
    print("Evaluating initial population...")
    for i in tqdm(range(x.shape[0]), total=x.shape[0]):
        instance = x[i]
        f.append(ea.evaluate(instance))
    print("Evaluation initial population completed")
    return np.asarray(f)

def print_best(x, activation_functions, batch_sizes, fitness):
    print(f"\nMost suitable parameters -- Fitness of {fitness}:")
    print(f"\tActivation function:           \t{activation_functions[x[0]]}")
    print(f"\tNodes layer 1:                 \t{x[1]}")
    print(f"\tNodes layer 2:                 \t{x[2]}")
    print(f"\tNodes layer 3:                 \t{x[3]}")
    print(f"\tSize encoding layer:           \t{x[4]}")
    print(f"\tBatch Normalization:           \t{x[5]}")
    print(f"\tBatch Size:                    \t{batch_sizes[x[6]]}")

def extract_population(population, activation_functions, batch_size):
    with open('AE output/optimization_output_15_04_2022.csv', mode='a', newline='') as output_file:
        csv_writer = csv.writer(output_file, delimiter=',', quotechar='"')
        for instance in population:
            row = []
            activation_function = activation_functions[instance[0]]

            nodes1 = instance[1]
            nodes2 = instance[2]
            nodes3 = instance[3]

            encoding_size = instance[4]

            batch_normalization = instance[5]
            batch_size = batch_sizes[instance[6]]
            csv_writer.writerow([activation_function, nodes1, nodes2, nodes3, encoding_size, batch_normalization, batch_size])

def plot_convergence(f_best):
    fig1 = make_subplots(rows=1, cols=1, specs=[[{'type':'xy'}]])
    
    x_values = []
    for i in range(len(f_best)):
        x_values.append(i)
    fig1.add_trace(go.Scatter(x=x_values, y=f_best, mode="lines"), row=1, col=1)

    fig1.update_layout(
        title = f'Fitness Over Autoencoder Tuning Generations', 
        xaxis1 = dict(title_text = 'Generation'),
        yaxis1 = dict(title_text = "Fitness")
    )
    fig1.write_image("Plots/opt autoencoder.png")
    fig1.show()

def validate_best(x, ea):
    print("\nValidating solution...")
    ea.evaluate(x)
    print("Solution validated")

In [29]:
"""
    instance = [activation function, --> one of list
        nodes layer 1, --> integer, not 0
        nodes layer 2, --> integer, may be 0
        nodes layer 3, --> integer, may be 0
        batch_normalization, --> binary integer
        batch_size --> one of list
        ]
"""

lookback = 5
val_year = 2018
test_year = 2019

files = {
    # varname: filename
    "S&P500": "Dataset v3/SP500_combined_data_20220412.csv",
}
for file in files:
    df = retrieve_data(file, files[file])

df = create_classification_data(df, lookback)

x_train, y_train, x_val, y_val, x_test, y_test = create_train_val_test(df, val_year, test_year)

x_train = scale_data(x_train)
x_val = scale_data(x_val)
x_test = scale_data(x_test)

variables = len(x_train.columns)
print(f"Input size: {variables}")
population_size = 20
generations = 10
epochs = 40
activation_functions = ['ReLU', 'LeakyReLU', 'sigmoid', 'tanh']
batch_sizes = [32, 64, 128, 256]

ea = EA(population_size, activation_functions, batch_sizes, variables)
x = init_population(population_size, activation_functions, batch_sizes, variables)
f = evaluate_init_population(ea, x)

populations = []
populations.append(x)
f_best = [f.min()]

start_time = datetime.now()

print("--> STARTING EVOLUTION")
early_stop = 0
for i in range(generations):
    print(f'Generation: {i}\tBest fitness: {f.min()}')
    x, f = ea.step(x, f)
    print(x)
    populations.append(x)

    if f.min() < f_best[-1]:
        f_best.append(f.min())
        early_stop = 0
    else:
        f_best.append(f_best[-1])
        early_stop += 1
    extract_population(x, activation_functions, batch_sizes)
    if early_stop == 3:
        print("Early stop triggered at generation {i} after not improving fitness for three generations")
        break
print("--> EVOLUTION FINISHED")

end_time = datetime.now()
evolution_time = end_time - start_time
evolution_time_seconds = evolution_time.total_seconds()
print(f"\nElapsed time in minutes: {evolution_time_seconds/60}")

print(f)
print(f.min())
index_best_parameters = np.where(f == f.min())[0][0]
print(index_best_parameters)
print_best(x[index_best_parameters], activation_functions, batch_sizes, f.min())
validate_best(x[index_best_parameters], ea)
plot_convergence(f_best)

  0%|                                                    | 0/20 [00:00<?, ?it/s]

Input size: 1500
Creating initial population...
Initial population ready
Evaluating initial population...
x [2 2075 2278 1323 875 0 1]


  5%|██▏                                        | 1/20 [02:39<50:31, 159.57s/it]

x [2 1613 664 942 430 0 0]


 10%|████▎                                      | 2/20 [04:56<43:53, 146.32s/it]

...

mutated [2.0 1154.4 1620.2 688.0 451.4 1.0 -0.2]
candidate [2 1154 1620 760 451 1 0]
x [2 1154 1620 760 451 1 0]


 95%|███████████████████████████████████████▉  | 19/20 [41:31<02:05, 125.85s/it]

mutated [2.0 887.2 2370.8 1317.0 175.8 1.0 1.2]
candidate [2 887 2371 1317 176 1 1]
x [2 887 2371 1317 176 1 1]


100%|██████████████████████████████████████████| 20/20 [43:34<00:00, 130.73s/it]


[[2 1376 3120 571 199 1 2]
 [2 786 2299 1257 191 1 1]
 [2 2180 2756 1241 205 1 2]
 [2 887 2371 1317 176 1 1]
 [2 1237 1758 1327 302 1 1]
 [2 1376 3120 571 199 1 1]
 [2 1335 1691 1211 243 1 1]
 [2 1224 2920 1167 228 1 1]
 [2 761 2221 1211 243 1 1]
 [2 1376 3120 801 199 1 1]
 [2 1051 1801 1185 243 1 1]
 [2 467 1729 1295 218 1 0]
 [2 930 1826 797 404 1 1]
 [2 1441 3278 578 218 1 0]
 [2 1312 1613 1084 386 1 1]
 [2 1376 3120 571 199 1 0]
 [2 761 1613 1084 386 1 1]
 [2 2180 2756 1214 205 1 1]
 [2 473 1571 777 473 1 1]
 [2 496 1777 1270 264 1 0]]
--> EVOLUTION FINISHED

Elapsed time in minutes: 553.9306301666667
[0.00916579 0.00953716 0.01058074 0.01077103 0.01181471 0.01183131
 0.01253977 0.01260201 0.0126396  0.01269041 0.01342895 0.01400287
 0.01532083 0.01605247 0.01621543 0.01624137 0.01646277 0.01699676
 0.01705681 0.01776947]
0.009165789444497849
0

Most suitable parameters -- Fitness of 0.009165789444497849:
	Activation function:           	sigmoid
	Nodes layer 1:                 	137

In [31]:
plot_convergence(f_best)

In [38]:
def plot_ae_convergence(train_loss, val_loss):
    fig1 = make_subplots(rows=1, cols=1, specs=[[{'type':'xy'}]])
    
    x_values = []
    for i in range(len(train_loss)):
        x_values.append(i)
    fig1.add_trace(go.Scatter(x=x_values, y=train_loss, name="Training Loss", mode="lines"), row=1, col=1)
    fig1.add_trace(go.Scatter(x=x_values, y=val_loss, name="Validation Loss", mode="lines"), row=1, col=1)

    fig1.update_layout(
        title = f'Training and Validation Loss Convergence', 
        xaxis1 = dict(title_text = 'Epoch'),
        yaxis1 = dict(title_text = "Loss")
    )
    fig1.write_image("Plots/autoencoder 2 convergence 100 epochs.png")
    fig1.show()

activation_function = "sigmoid"
nodes1 = 1376
nodes2 = 3120
nodes3 = 571
encoding_size = 199

nodes1 = 1592
nodes2 = 1897
nodes3 = 1323
encoding_size = 1125

epochs = 100
batch_size = 128
batch_normalization = 1

input_shape = x_train.shape[1]
visible = Input(shape=(input_shape,))

# ENCODER
# encoder layer 1
encoder = Dense(nodes1)(visible)
# apply batch normalization
encoder = BatchNormalization()(encoder)
# activate layer
encoder = sigmoid(encoder)

# encoder layer 2
encoder = Dense(nodes2)(encoder)
# apply batch normalization
encoder = BatchNormalization()(encoder)
# activate layer
encoder = sigmoid(encoder)

# encoder layer 3
encoder = Dense(nodes3)(encoder)
# apply batch normalization
encoder = BatchNormalization()(encoder)
# activate layer
encoder = sigmoid(encoder)

# ENCODING
encoding = Dense(encoding_size)(encoder) 

# DECODER
# decoder layer 1
decoder = Dense(nodes3)(encoding)
# apply batch normalization
decoder = BatchNormalization()(decoder)
# activate layer
decoder = sigmoid(decoder)

# decoder layer 2
decoder = Dense(nodes2)(decoder)
# apply batch normalization
decoder = BatchNormalization()(decoder)
# activate layer
decoder = sigmoid(decoder)

# decoder layer 3
decoder = Dense(nodes1)(decoder)
# apply batch normalization
decoder = BatchNormalization()(decoder)
# activate layer
decoder = sigmoid(decoder)

output = Dense(input_shape, activation='linear')(decoder)

model = Model(inputs=visible, outputs=output)
model.compile(optimizer='adam', loss='mse')

history = model.fit(x_train, x_train, 
            epochs=epochs, batch_size=batch_size, verbose=2, 
            validation_data=(x_val, x_val))

train_loss = history.history['loss']
val_loss = history.history['val_loss']

plot_ae_convergence(train_loss, val_loss)

Epoch 1/100
21/21 - 5s - loss: 0.1260 - val_loss: 0.0785
Epoch 2/100
21/21 - 3s - loss: 0.0184 - val_loss: 0.0589
Epoch 3/100
21/21 - 3s - loss: 0.0091 - val_loss: 0.0536
Epoch 4/100
21/21 - 2s - loss: 0.0072 - val_loss: 0.0507
Epoch 5/100
21/21 - 3s - loss: 0.0064 - val_loss: 0.0483
Epoch 6/100
21/21 - 3s - loss: 0.0060 - val_loss: 0.0461
Epoch 7/100
21/21 - 3s - loss: 0.0055 - val_loss: 0.0435
Epoch 8/100
21/21 - 3s - loss: 0.0052 - val_loss: 0.0435
Epoch 9/100
21/21 - 3s - loss: 0.0048 - val_loss: 0.0399
Epoch 10/100
21/21 - 3s - loss: 0.0044 - val_loss: 0.0378
Epoch 11/100
21/21 - 3s - loss: 0.0042 - val_loss: 0.0360
Epoch 12/100
21/21 - 4s - loss: 0.0042 - val_loss: 0.0341
Epoch 13/100
21/21 - 3s - loss: 0.0042 - val_loss: 0.0320
Epoch 14/100
21/21 - 3s - loss: 0.0039 - val_loss: 0.0291
Epoch 15/100
21/21 - 3s - loss: 0.0039 - val_loss: 0.0290
Epoch 16/100
21/21 - 3s - loss: 0.0036 - val_loss: 0.0280
Epoch 17/100
21/21 - 3s - loss: 0.0036 - val_loss: 0.0265
Epoch 18/100
21/21 - 3s